In [5]:
from spafe.features.bfcc import bfcc
from spafe.features.lpc import lpc
from spafe.features.pncc import pncc
from spafe.features.rplp import rplp
from spafe.features.ngcc import ngcc
from spafe.features.msrcc import msrcc

In [4]:
import soundfile as sf
from pydub import AudioSegment
import librosa
import numpy as np 
import os
import fnmatch
import numpy as np
import librosa
import torch # import PyTorch
import torch
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

In [3]:
from spafe.features.gfcc import gfcc
from spafe.utils import vis
import scipy.io.wavfile as wav

In [2]:

directory = 'F:\dataset_sri'



output_directory3 = 'F:\data_bfcc'
output_directory4 = 'F:\data_pncc'

output_directory7 = 'F:\data_ngcc'
output_directory8 =  'F:\data_msrcc'


In [6]:

import scipy.io.wavfile as wav

os.makedirs(output_directory3, exist_ok=True)
os.makedirs(output_directory4, exist_ok=True)


os.makedirs(output_directory7, exist_ok=True)
os.makedirs(output_directory8, exist_ok=True)

wav_files = [os.path.join(root, file)
             for root, dirs, files in os.walk(directory)
             for file in fnmatch.filter(files, '*.m4a')]


def extract_features(file_name):
    # Convert m4a file to wav            
    audio = AudioSegment.from_file(file_name)
    audio.export("temp.wav", format="wav")
    (samplerate, data) = wav.read("temp.wav")
    
    bfccs = bfcc(sig=data, fs=samplerate, num_ceps=13)
    
    pnccs = pncc(sig=data, fs=samplerate, num_ceps=13)
   
    ngccs = ngcc(sig=data, fs=samplerate, num_ceps=13)
    msrccs = msrcc(sig=data, fs=samplerate, num_ceps=13)
    
    # Compute the mean along the time axis (axis 0) to get a single feature vector for each file
    bfccs_processed = np.mean(bfccs, axis=0)
  
    pnccs_processed = np.mean(pnccs, axis=0)
    
    ngccs_processed = np.mean(ngccs, axis=0)
    msrccs_processed = np.mean(msrccs, axis=0)
    
    return bfccs_processed, pnccs_processed, ngccs_processed, msrccs_processed

for file in wav_files:
    bfccs,  pnccs, ngccs, msrccs = extract_features(file)
    
    # Convert arrays to torch tensors
    bfccs = torch.tensor(bfccs)
    
    pnccs = torch.tensor(pnccs)
    
    ngccs = torch.tensor(ngccs)
    msrccs = torch.tensor(msrccs)

    # Construct output file names by replacing .wav with .pt and changing the directory
    output_file_bfcc = os.path.join(output_directory3, os.path.basename(file).replace('.m4a', '_bfcc.pt'))
    
    output_file_pncc = os.path.join(output_directory4, os.path.basename(file).replace('.m4a', '_pncc.pt'))
   
    output_file_ngcc = os.path.join(output_directory7, os.path.basename(file).replace('.m4a', '_ngcc.pt'))
    output_file_msrcc = os.path.join(output_directory8, os.path.basename(file).replace('.m4a', '_msrcc.pt'))
    
    torch.save(bfccs, output_file_bfcc)
    
    torch.save(pnccs, output_file_pncc)
    
    torch.save(ngccs, output_file_ngcc)
    torch.save(msrccs, output_file_msrcc)









c:\Users\Dell\AppData\Local\Programs\Python\Python39\lib\site-packages\spafe\features\pncc.py:169: RuntimeWarning: divide by zero encountered in double_scalars
  [R_tilde[m, lprime] / Q_tilde[m, lprime] for lprime in range(l1, l2)]
c:\Users\Dell\AppData\Local\Programs\Python\Python39\lib\site-packages\spafe\features\pncc.py:437: RuntimeWarning: invalid value encountered in multiply
  T = P * S_tilde
c:\Users\Dell\AppData\Local\Programs\Python\Python39\lib\site-packages\spafe\features\msrcc.py:153: RuntimeWarning: divide by zero encountered in power
  features = features**gamma
c:\Users\Dell\AppData\Local\Programs\Python\Python39\lib\site-packages\spafe\features\pncc.py:169: RuntimeWarning: invalid value encountered in double_scalars
  [R_tilde[m, lprime] / Q_tilde[m, lprime] for lprime in range(l1, l2)]


In [7]:
import numpy as np
import scipy.io.wavfile as wav
import os
import fnmatch
import librosa
from pydub import AudioSegment
import torch

directory = r'F:\dataset_sri'
output_directory= r'F:\data_mfmgdcc'
os.makedirs(output_directory, exist_ok=True)

wav_files = [os.path.join(root, file)
             for root, dirs, files in os.walk(directory)
             for file in fnmatch.filter(files, '*.m4a')]

def compute_mel_spectrogram(data, samplerate, n_fft=512, hop_length=256, n_mels=40):
    # Compute the Mel spectrogram
    mel_spectrogram = librosa.feature.melspectrogram(data, sr=samplerate, n_fft=n_fft, hop_length=hop_length, n_mels=n_mels)

    return mel_spectrogram

def calculate_mfmgdcc(mel_spectrogram):
    # Calculate the phase of the Mel spectrogram
    phase = np.unwrap(np.angle(mel_spectrogram))

    # Compute the group delay function
    group_delay = np.diff(phase, axis=-1) / np.diff(mel_spectrogram, axis=-1)
    group_delay = np.pad(group_delay, ((0, 0), (0, 1)), mode='constant')

    # Compute the modified group delay function
    mgd = mel_spectrogram * group_delay

    # Take the DCT of the MGDF to get MF-MGDCCs
    mfmgdcc = scipy.fftpack.dct(mgd, axis=-1)

    # Mean normalization
    mfmgdcc_processed = np.mean(mfmgdcc, axis=0)

    return mfmgdcc_processed

def extract_mfmgdcc(file_name):
    # Convert m4a file to wav            
    audio = AudioSegment.from_file(file_name)
    audio.export("temp.wav", format="wav")
    (samplerate, data) = wav.read("temp.wav")
    
    # Convert to floating point values ranging from -1 to 1
    data = data / 32767.0
    
    mel_spectrogram = compute_mel_spectrogram(data, samplerate)
    mfmgdccs = calculate_mfmgdcc(mel_spectrogram)
    
    return mfmgdccs

for file in wav_files:
    mfmgdccs = extract_mfmgdcc(file)
    
    # Convert array to torch tensor
    mfmgdccs = torch.tensor(mfmgdccs)

    # Construct output file name by replacing .m4a with .pt and changing the directory
    output_file_mfmgdcc = os.path.join(output_directory, os.path.basename(file).replace('.m4a', '_mfmgdcc.pt'))
    
    # Save to a PyTorch file (.pt)
    torch.save(mfmgdccs, output_file_mfmgdcc)


TypeError: melspectrogram() takes 0 positional arguments but 1 positional argument (and 3 keyword-only arguments) were given

In [9]:
import numpy as np
import scipy.io.wavfile as wav
import os
import fnmatch
import librosa
from pydub import AudioSegment
import torch
import scipy

directory = r'F:\dataset_sri'
output_directory= r'F:\data_mfmgdcc'
os.makedirs(output_directory, exist_ok=True)

wav_files = [os.path.join(root, file)
             for root, dirs, files in os.walk(directory)
             for file in fnmatch.filter(files, '*.m4a')]

def compute_mel_spectrogram(data, samplerate, n_fft=512, hop_length=256, n_mels=40):
    print(f"Shape of the data: {data.shape}")  # Debugging line
    mel_spectrogram = librosa.feature.melspectrogram(y=data, sr=samplerate, n_fft=n_fft, hop_length=hop_length, n_mels=n_mels)
    return mel_spectrogram

def calculate_mfmgdcc(mel_spectrogram):
    phase = np.unwrap(np.angle(mel_spectrogram))
    group_delay = np.diff(phase, axis=-1) / np.diff(mel_spectrogram, axis=-1)
    group_delay = np.pad(group_delay, ((0, 0), (0, 1)), mode='constant')
    mgd = mel_spectrogram * group_delay
    mfmgdcc = scipy.fftpack.dct(mgd, axis=-1)
    mfmgdcc_processed = np.mean(mfmgdcc, axis=0)
    return mfmgdcc_processed

def extract_mfmgdcc(file_name):
    audio = AudioSegment.from_file(file_name)
    audio.export("temp.wav", format="wav")
    (samplerate, data) = wav.read("temp.wav")
    data = data / 32767.0
    mel_spectrogram = compute_mel_spectrogram(data, samplerate)
    mfmgdccs = calculate_mfmgdcc(mel_spectrogram)
    return mfmgdccs

for file in wav_files:
    mfmgdccs = extract_mfmgdcc(file)
    mfmgdccs = torch.tensor(mfmgdccs)
    output_file_mfmgdcc = os.path.join(output_directory, os.path.basename(file).replace('.m4a', '_mfmgdcc.pt'))
    torch.save(mfmgdccs, output_file_mfmgdcc)


Shape of the data: (181248,)


C:\Users\Dell\AppData\Local\Temp\ipykernel_1676\1660773744.py:25: RuntimeWarning: invalid value encountered in divide
  group_delay = np.diff(phase, axis=-1) / np.diff(mel_spectrogram, axis=-1)


Shape of the data: (74752,)
Shape of the data: (66560,)
Shape of the data: (109568,)
Shape of the data: (294912,)
Shape of the data: (107520,)
Shape of the data: (186368,)
Shape of the data: (243712,)
Shape of the data: (179200,)
Shape of the data: (133120,)
Shape of the data: (78848,)
Shape of the data: (83968,)
Shape of the data: (79872,)
Shape of the data: (138240,)
Shape of the data: (175104,)
Shape of the data: (92160,)
Shape of the data: (107520,)
Shape of the data: (86016,)
Shape of the data: (77824,)
Shape of the data: (77824,)
Shape of the data: (117760,)
Shape of the data: (87040,)
Shape of the data: (86016,)
Shape of the data: (95232,)
Shape of the data: (73728,)
Shape of the data: (109568,)
Shape of the data: (219136,)
Shape of the data: (145408,)
Shape of the data: (92160,)
Shape of the data: (150528,)
Shape of the data: (136192,)
Shape of the data: (132096,)
Shape of the data: (95232,)
Shape of the data: (81920,)
Shape of the data: (90112,)
Shape of the data: (107520,)
Sh

In [11]:
import os
import torch
import torch.nn as nn
from torch.autograd import Variable

directory = r'F:\data_whisper'
output_directory= r'F:\data_whisper2'
class FeatureReducer(nn.Module):
    def __init__(self):
        super(FeatureReducer, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d((1, None))

    def forward(self, x):
        return self.avg_pool(x).view(x.size(0), -1)

reducer = FeatureReducer()



# Get list of .pt files in the directory
files = [f for f in os.listdir(directory) if f.endswith('.pt')]

# Loop over each file
for file_name in files:
    file_path = os.path.join(directory, file_name)

    # Load the tensor
    tensor = torch.load(file_path)

    # Reshape the tensor to (batch size, sequence_length, input_dim)
    tensor = tensor.view(1, 1500, 384)

    # Pass the tensor through the reducer
    reduced_tensor = reducer(tensor)

    # Save the reduced tensor
    output_file_path = os.path.join(output_directory, "reduced_" + file_name)
    torch.save(reduced_tensor, output_file_path)


In [15]:



import os
import torch
import torch.nn as nn
from torch.autograd import Variable

directory = r'F:\data_whisper'
output_directory= r'F:\data_whisper3'

class FeatureReducer(nn.Module):
    def __init__(self):
        super(FeatureReducer, self).__init__()
        self.conv1 = nn.Conv2d(1, 64, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(64, 1, kernel_size=3, stride=1, padding=1)
        self.avg_pool = nn.AdaptiveAvgPool2d((25, 25))

    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = self.conv2(x)
        return self.avg_pool(x)

reducer = FeatureReducer()



# Get list of .pt files in the directory
files = [f for f in os.listdir(directory) if f.endswith('.pt')]

# Loop over each file
for file_name in files:
    file_path = os.path.join(directory, file_name)

    # Load the tensor
    tensor = torch.load(file_path)

    # Reshape the tensor to (batch size, sequence_length, input_dim)
    tensor = tensor.view(1, 1500, 384)

    # Pass the tensor through the reducer
    reduced_tensor = reducer(tensor)

    # Save the reduced tensor
    output_file_path = os.path.join(output_directory, "reduced_" + file_name)
    torch.save(reduced_tensor, output_file_path)


In [2]:
import os
import numpy as np
import torch
from keras.models import Sequential
from keras.layers import Bidirectional, LSTM, Dropout, Dense
from sklearn.model_selection import train_test_split

directory = r'F:\data_whisper3'  # Replace with the directory path containing the .pt files

data = []
labels = []

for i in range(5993):
    filename = f'reduced_{i}_whisper.pt'
    if filename in os.listdir(directory):  # Ensure the file exists in the directory
        feature = torch.load(os.path.join(directory, filename)).requires_grad_(False)
        print(filename)
        data.append(feature.view(-1, 1).numpy())
        labels.append(1 if i <= 2310 else 0)

labels = np.array(labels)
data  = np.array(data)
num_classes = len(np.unique(labels))
print("Shape of the first reshaped item in data:", data[0].shape)

X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

print("Shape of labels: ", labels.shape)  # It should be (n_samples, n_classes)

def get_model(op_nodes):
    model = Sequential()
    model.add(Bidirectional(LSTM(units=32, return_sequences=True), input_shape=(data[0].shape[0], data[0].shape[1])))
    model.add(Dropout(0.1))
    model.add(Bidirectional(LSTM(units=32)))
    model.add(Dropout(0.1))
    model.add(Dense(op_nodes, activation='softmax'))

    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    return model

# Create the BiLSTM model
model = get_model(num_classes)

# Train the model
model.fit(X_train, y_train, batch_size=32, epochs=10, verbose=1, validation_data=(X_test, y_test))

# Evaluate the model
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])


reduced_0_whisper.pt
reduced_1_whisper.pt
reduced_2_whisper.pt
reduced_3_whisper.pt
reduced_4_whisper.pt
reduced_5_whisper.pt
reduced_6_whisper.pt
reduced_7_whisper.pt
reduced_8_whisper.pt
reduced_9_whisper.pt
reduced_10_whisper.pt
reduced_11_whisper.pt
reduced_12_whisper.pt
reduced_13_whisper.pt
reduced_14_whisper.pt
reduced_15_whisper.pt
reduced_16_whisper.pt
reduced_17_whisper.pt
reduced_18_whisper.pt
reduced_19_whisper.pt
reduced_20_whisper.pt
reduced_21_whisper.pt
reduced_22_whisper.pt
reduced_23_whisper.pt
reduced_24_whisper.pt
reduced_25_whisper.pt
reduced_26_whisper.pt
reduced_27_whisper.pt
reduced_28_whisper.pt
reduced_29_whisper.pt
reduced_30_whisper.pt
reduced_31_whisper.pt
reduced_32_whisper.pt
reduced_33_whisper.pt
reduced_34_whisper.pt
reduced_35_whisper.pt
reduced_36_whisper.pt
reduced_37_whisper.pt
reduced_38_whisper.pt
reduced_39_whisper.pt
reduced_40_whisper.pt
reduced_41_whisper.pt
reduced_42_whisper.pt
reduced_43_whisper.pt
reduced_44_whisper.pt
reduced_45_whisper.p

In [2]:
import os
import numpy as np
import torch
import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from sklearn.model_selection import train_test_split
from keras.layers import Conv1D, MaxPooling1D

directory = r'F:\data_bfcc'  # Replace with the directory path containing the .pt files

data = []
labels = []



for i in range(5993):
    filename = f'{i}_bfcc.pt'
    if filename in os.listdir(directory):  # Ensure the file exists in the directory
        feature = torch.load(os.path.join(directory, filename))
        print(filename)
        # Reshape the feature to a 2D array with a single row and 40 columns
        data.append(feature.view(1, -1).numpy())
        # Assign labels: first 2312 files are labelled 1, rest are labelled 0
        labels.append(1 if i <= 2310 else 0)


labels = np.array(labels)
data  = np.array(data)

# Now you should be able to get a 3D shape from each element in data
print("Shape of the first reshaped item in data:", data[0].shape)

# Split the data into training and testing sets
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

# Reshape the input data to match the CNN input shape
input_shape = (data[0].shape[1], 1)

X_train = X_train.reshape(X_train.shape[0], X_train.shape[2], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[2], 1)
# Convert labels to one-hot encoding
num_classes = len(np.unique(labels))
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

# Now you can define and train your CNN as before
# ...




# Create the CNN model


model = Sequential()
model.add(Conv1D(32, kernel_size=3, activation='relu', input_shape=input_shape))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(64, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

# Compile and train the model
model.compile(loss=keras.losses.categorical_crossentropy, optimizer=keras.optimizers.Adam(), metrics=['accuracy'])
model.fit(X_train, y_train, batch_size=32, epochs=10, verbose=1, validation_data=(X_test, y_test))

# Evaluate the model
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])


0_bfcc.pt
1_bfcc.pt
2_bfcc.pt
3_bfcc.pt
4_bfcc.pt
5_bfcc.pt
6_bfcc.pt
7_bfcc.pt
8_bfcc.pt
9_bfcc.pt
10_bfcc.pt
11_bfcc.pt
12_bfcc.pt
13_bfcc.pt
14_bfcc.pt
15_bfcc.pt
16_bfcc.pt
17_bfcc.pt
18_bfcc.pt
19_bfcc.pt
20_bfcc.pt
21_bfcc.pt
22_bfcc.pt
23_bfcc.pt
24_bfcc.pt
25_bfcc.pt
26_bfcc.pt
27_bfcc.pt
28_bfcc.pt
29_bfcc.pt
30_bfcc.pt
31_bfcc.pt
32_bfcc.pt
33_bfcc.pt
34_bfcc.pt
35_bfcc.pt
36_bfcc.pt
37_bfcc.pt
38_bfcc.pt
39_bfcc.pt
40_bfcc.pt
41_bfcc.pt
42_bfcc.pt
43_bfcc.pt
44_bfcc.pt
45_bfcc.pt
46_bfcc.pt
47_bfcc.pt
48_bfcc.pt
49_bfcc.pt
50_bfcc.pt
51_bfcc.pt
52_bfcc.pt
53_bfcc.pt
54_bfcc.pt
55_bfcc.pt
56_bfcc.pt
57_bfcc.pt
58_bfcc.pt
59_bfcc.pt
60_bfcc.pt
61_bfcc.pt
62_bfcc.pt
63_bfcc.pt
64_bfcc.pt
65_bfcc.pt
66_bfcc.pt
67_bfcc.pt
68_bfcc.pt
69_bfcc.pt
70_bfcc.pt
71_bfcc.pt
72_bfcc.pt
73_bfcc.pt
74_bfcc.pt
75_bfcc.pt
76_bfcc.pt
77_bfcc.pt
78_bfcc.pt
79_bfcc.pt
80_bfcc.pt
81_bfcc.pt
82_bfcc.pt
83_bfcc.pt
84_bfcc.pt
85_bfcc.pt
86_bfcc.pt
87_bfcc.pt
88_bfcc.pt
89_bfcc.pt
90_bfcc.pt
91_bfcc.p

In [3]:
import os
import numpy as np
import torch
import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from sklearn.model_selection import train_test_split
from keras.layers import Conv1D, MaxPooling1D

directory = r'F:\data_msrcc'  # Replace with the directory path containing the .pt files

data = []
labels = []



for i in range(5993):
    filename = f'{i}_msrcc.pt'
    if filename in os.listdir(directory):  # Ensure the file exists in the directory
        feature = torch.load(os.path.join(directory, filename))
        print(filename)
        # Reshape the feature to a 2D array with a single row and 40 columns
        data.append(feature.view(1, -1).numpy())
        # Assign labels: first 2312 files are labelled 1, rest are labelled 0
        labels.append(1 if i <= 2310 else 0)


labels = np.array(labels)
data  = np.array(data)

# Now you should be able to get a 3D shape from each element in data
print("Shape of the first reshaped item in data:", data[0].shape)

# Split the data into training and testing sets
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

# Reshape the input data to match the CNN input shape
input_shape = (data[0].shape[1], 1)

X_train = X_train.reshape(X_train.shape[0], X_train.shape[2], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[2], 1)
# Convert labels to one-hot encoding
num_classes = len(np.unique(labels))
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

# Now you can define and train your CNN as before
# ...




# Create the CNN model


model = Sequential()
model.add(Conv1D(32, kernel_size=3, activation='relu', input_shape=input_shape))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(64, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

# Compile and train the model
model.compile(loss=keras.losses.categorical_crossentropy, optimizer=keras.optimizers.Adam(), metrics=['accuracy'])
model.fit(X_train, y_train, batch_size=32, epochs=10, verbose=1, validation_data=(X_test, y_test))

# Evaluate the model
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])


0_msrcc.pt
1_msrcc.pt
2_msrcc.pt
3_msrcc.pt
4_msrcc.pt
5_msrcc.pt
6_msrcc.pt
7_msrcc.pt
8_msrcc.pt
9_msrcc.pt
10_msrcc.pt
11_msrcc.pt
12_msrcc.pt
13_msrcc.pt
14_msrcc.pt
15_msrcc.pt
16_msrcc.pt
17_msrcc.pt
18_msrcc.pt
19_msrcc.pt
20_msrcc.pt
21_msrcc.pt
22_msrcc.pt
23_msrcc.pt
24_msrcc.pt
25_msrcc.pt
26_msrcc.pt
27_msrcc.pt
28_msrcc.pt
29_msrcc.pt
30_msrcc.pt
31_msrcc.pt
32_msrcc.pt
33_msrcc.pt
34_msrcc.pt
35_msrcc.pt
36_msrcc.pt
37_msrcc.pt
38_msrcc.pt
39_msrcc.pt
40_msrcc.pt
41_msrcc.pt
42_msrcc.pt
43_msrcc.pt
44_msrcc.pt
45_msrcc.pt
46_msrcc.pt
47_msrcc.pt
48_msrcc.pt
49_msrcc.pt
50_msrcc.pt
51_msrcc.pt
52_msrcc.pt
53_msrcc.pt
54_msrcc.pt
55_msrcc.pt
56_msrcc.pt
57_msrcc.pt
58_msrcc.pt
59_msrcc.pt
60_msrcc.pt
61_msrcc.pt
62_msrcc.pt
63_msrcc.pt
64_msrcc.pt
65_msrcc.pt
66_msrcc.pt
67_msrcc.pt
68_msrcc.pt
69_msrcc.pt
70_msrcc.pt
71_msrcc.pt
72_msrcc.pt
73_msrcc.pt
74_msrcc.pt
75_msrcc.pt
76_msrcc.pt
77_msrcc.pt
78_msrcc.pt
79_msrcc.pt
80_msrcc.pt
81_msrcc.pt
82_msrcc.pt
83_msrcc.pt
84

In [4]:
import os
import numpy as np
import torch
import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from sklearn.model_selection import train_test_split
from keras.layers import Conv1D, MaxPooling1D

directory = r'F:\data_ngcc'  # Replace with the directory path containing the .pt files

data = []
labels = []



for i in range(5993):
    filename = f'{i}_ngcc.pt'
    if filename in os.listdir(directory):  # Ensure the file exists in the directory
        feature = torch.load(os.path.join(directory, filename))
        print(filename)
        # Reshape the feature to a 2D array with a single row and 40 columns
        data.append(feature.view(1, -1).numpy())
        # Assign labels: first 2312 files are labelled 1, rest are labelled 0
        labels.append(1 if i <= 2310 else 0)


labels = np.array(labels)
data  = np.array(data)

# Now you should be able to get a 3D shape from each element in data
print("Shape of the first reshaped item in data:", data[0].shape)

# Split the data into training and testing sets
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

# Reshape the input data to match the CNN input shape
input_shape = (data[0].shape[1], 1)

X_train = X_train.reshape(X_train.shape[0], X_train.shape[2], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[2], 1)
# Convert labels to one-hot encoding
num_classes = len(np.unique(labels))
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

# Now you can define and train your CNN as before
# ...




# Create the CNN model


model = Sequential()
model.add(Conv1D(32, kernel_size=3, activation='relu', input_shape=input_shape))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(64, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

# Compile and train the model
model.compile(loss=keras.losses.categorical_crossentropy, optimizer=keras.optimizers.Adam(), metrics=['accuracy'])
model.fit(X_train, y_train, batch_size=32, epochs=10, verbose=1, validation_data=(X_test, y_test))

# Evaluate the model
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])


0_ngcc.pt
1_ngcc.pt
2_ngcc.pt
3_ngcc.pt
4_ngcc.pt
5_ngcc.pt
6_ngcc.pt
7_ngcc.pt
8_ngcc.pt
9_ngcc.pt
10_ngcc.pt
11_ngcc.pt
12_ngcc.pt
13_ngcc.pt
14_ngcc.pt
15_ngcc.pt
16_ngcc.pt
17_ngcc.pt
18_ngcc.pt
19_ngcc.pt
20_ngcc.pt
21_ngcc.pt
22_ngcc.pt
23_ngcc.pt
24_ngcc.pt
25_ngcc.pt
26_ngcc.pt
27_ngcc.pt
28_ngcc.pt
29_ngcc.pt
30_ngcc.pt
31_ngcc.pt
32_ngcc.pt
33_ngcc.pt
34_ngcc.pt
35_ngcc.pt
36_ngcc.pt
37_ngcc.pt
38_ngcc.pt
39_ngcc.pt
40_ngcc.pt
41_ngcc.pt
42_ngcc.pt
43_ngcc.pt
44_ngcc.pt
45_ngcc.pt
46_ngcc.pt
47_ngcc.pt
48_ngcc.pt
49_ngcc.pt
50_ngcc.pt
51_ngcc.pt
52_ngcc.pt
53_ngcc.pt
54_ngcc.pt
55_ngcc.pt
56_ngcc.pt
57_ngcc.pt
58_ngcc.pt
59_ngcc.pt
60_ngcc.pt
61_ngcc.pt
62_ngcc.pt
63_ngcc.pt
64_ngcc.pt
65_ngcc.pt
66_ngcc.pt
67_ngcc.pt
68_ngcc.pt
69_ngcc.pt
70_ngcc.pt
71_ngcc.pt
72_ngcc.pt
73_ngcc.pt
74_ngcc.pt
75_ngcc.pt
76_ngcc.pt
77_ngcc.pt
78_ngcc.pt
79_ngcc.pt
80_ngcc.pt
81_ngcc.pt
82_ngcc.pt
83_ngcc.pt
84_ngcc.pt
85_ngcc.pt
86_ngcc.pt
87_ngcc.pt
88_ngcc.pt
89_ngcc.pt
90_ngcc.pt
91_ngcc.p

In [5]:
import os
import numpy as np
import torch
import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from sklearn.model_selection import train_test_split
from keras.layers import Conv1D, MaxPooling1D

directory = r'F:\data_pncc'  # Replace with the directory path containing the .pt files

data = []
labels = []



for i in range(5993):
    filename = f'{i}_pncc.pt'
    if filename in os.listdir(directory):  # Ensure the file exists in the directory
        feature = torch.load(os.path.join(directory, filename))
        print(filename)
        # Reshape the feature to a 2D array with a single row and 40 columns
        data.append(feature.view(1, -1).numpy())
        # Assign labels: first 2312 files are labelled 1, rest are labelled 0
        labels.append(1 if i <= 2310 else 0)


labels = np.array(labels)
data  = np.array(data)

# Now you should be able to get a 3D shape from each element in data
print("Shape of the first reshaped item in data:", data[0].shape)

# Split the data into training and testing sets
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

# Reshape the input data to match the CNN input shape
input_shape = (data[0].shape[1], 1)

X_train = X_train.reshape(X_train.shape[0], X_train.shape[2], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[2], 1)
# Convert labels to one-hot encoding
num_classes = len(np.unique(labels))
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

# Now you can define and train your CNN as before
# ...




# Create the CNN model


model = Sequential()
model.add(Conv1D(32, kernel_size=3, activation='relu', input_shape=input_shape))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(64, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

# Compile and train the model
model.compile(loss=keras.losses.categorical_crossentropy, optimizer=keras.optimizers.Adam(), metrics=['accuracy'])
model.fit(X_train, y_train, batch_size=32, epochs=10, verbose=1, validation_data=(X_test, y_test))

# Evaluate the model
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])


0_pncc.pt
1_pncc.pt
2_pncc.pt
3_pncc.pt
4_pncc.pt
5_pncc.pt
6_pncc.pt
7_pncc.pt
8_pncc.pt
9_pncc.pt
10_pncc.pt
11_pncc.pt
12_pncc.pt
13_pncc.pt
14_pncc.pt
15_pncc.pt
16_pncc.pt
17_pncc.pt
18_pncc.pt
19_pncc.pt
20_pncc.pt
21_pncc.pt
22_pncc.pt
23_pncc.pt
24_pncc.pt
25_pncc.pt
26_pncc.pt
27_pncc.pt
28_pncc.pt
29_pncc.pt
30_pncc.pt
31_pncc.pt
32_pncc.pt
33_pncc.pt
34_pncc.pt
35_pncc.pt
36_pncc.pt
37_pncc.pt
38_pncc.pt
39_pncc.pt
40_pncc.pt
41_pncc.pt
42_pncc.pt
43_pncc.pt
44_pncc.pt
45_pncc.pt
46_pncc.pt
47_pncc.pt
48_pncc.pt
49_pncc.pt
50_pncc.pt
51_pncc.pt
52_pncc.pt
53_pncc.pt
54_pncc.pt
55_pncc.pt
56_pncc.pt
57_pncc.pt
58_pncc.pt
59_pncc.pt
60_pncc.pt
61_pncc.pt
62_pncc.pt
63_pncc.pt
64_pncc.pt
65_pncc.pt
66_pncc.pt
67_pncc.pt
68_pncc.pt
69_pncc.pt
70_pncc.pt
71_pncc.pt
72_pncc.pt
73_pncc.pt
74_pncc.pt
75_pncc.pt
76_pncc.pt
77_pncc.pt
78_pncc.pt
79_pncc.pt
80_pncc.pt
81_pncc.pt
82_pncc.pt
83_pncc.pt
84_pncc.pt
85_pncc.pt
86_pncc.pt
87_pncc.pt
88_pncc.pt
89_pncc.pt
90_pncc.pt
91_pncc.p

In [7]:
import os
import numpy as np
import torch
import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from sklearn.model_selection import train_test_split
from keras.layers import Conv1D, MaxPooling1D

directory = r'F:\data_mgdcc'  # Replace with the directory path containing the .pt files

data = []
labels = []



for i in range(5993):
    filename = f'{i}_mgdcc.pt'
    if filename in os.listdir(directory):  # Ensure the file exists in the directory
        feature = torch.load(os.path.join(directory, filename))
        print(filename)
        # Reshape the feature to a 2D array with a single row and 40 columns
        data.append(feature.view(1, -1).numpy())
        # Assign labels: first 2312 files are labelled 1, rest are labelled 0
        labels.append(1 if i <= 2310 else 0)


labels = np.array(labels)
data  = np.array(data)

# Now you should be able to get a 3D shape from each element in data
print("Shape of the first reshaped item in data:", data[0].shape)

# Split the data into training and testing sets
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

# Reshape the input data to match the CNN input shape
input_shape = (data[0].shape[1], 1)

X_train = X_train.reshape(X_train.shape[0], X_train.shape[2], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[2], 1)
# Convert labels to one-hot encoding
num_classes = len(np.unique(labels))
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

# Now you can define and train your CNN as before
# ...




# Create the CNN model


model = Sequential()
model.add(Conv1D(32, kernel_size=3, activation='relu', input_shape=input_shape))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(64, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

# Compile and train the model
model.compile(loss=keras.losses.categorical_crossentropy, optimizer=keras.optimizers.Adam(), metrics=['accuracy'])
model.fit(X_train, y_train, batch_size=32, epochs=10, verbose=1, validation_data=(X_test, y_test))

0_mgdcc.pt
Skipping 1_mgdcc.pt due to incompatible shape: torch.Size([293])
1_mgdcc.pt
Skipping 2_mgdcc.pt due to incompatible shape: torch.Size([609])
2_mgdcc.pt
Skipping 3_mgdcc.pt due to incompatible shape: torch.Size([1701])
3_mgdcc.pt
Skipping 4_mgdcc.pt due to incompatible shape: torch.Size([713])
4_mgdcc.pt
Skipping 5_mgdcc.pt due to incompatible shape: torch.Size([261])
5_mgdcc.pt
Skipping 6_mgdcc.pt due to incompatible shape: torch.Size([413])
6_mgdcc.pt
Skipping 7_mgdcc.pt due to incompatible shape: torch.Size([293])
7_mgdcc.pt
Skipping 8_mgdcc.pt due to incompatible shape: torch.Size([673])
8_mgdcc.pt
Skipping 9_mgdcc.pt due to incompatible shape: torch.Size([341])
9_mgdcc.pt
Skipping 10_mgdcc.pt due to incompatible shape: torch.Size([261])
10_mgdcc.pt
Skipping 11_mgdcc.pt due to incompatible shape: torch.Size([253])
11_mgdcc.pt
Skipping 12_mgdcc.pt due to incompatible shape: torch.Size([253])
12_mgdcc.pt
Skipping 13_mgdcc.pt due to incompatible shape: torch.Size([477])
13_m

ValueError: Found input variables with inconsistent numbers of samples: [10, 5993]

In [11]:
import tensorflow

In [12]:
from tensorflow.keras.preprocessing.sequence import pad_sequences


In [14]:
import os
import numpy as np
import torch
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import categorical_crossentropy
from sklearn.model_selection import train_test_split

directory = r'F:\data_mgdcc'  # Replace with the directory path containing the .pt files

data = []
labels = []
max_length = 0

for i in range(5993):
    filename = f'{i}_mgdcc.pt'
    if filename in os.listdir(directory):  # Ensure the file exists in the directory
        feature = torch.load(os.path.join(directory, filename))
        feature = feature.view(-1)  # reshape to 1D tensor
        data.append(feature.numpy())
        max_length = max(max_length, feature.shape[0])
        print(filename)
        labels.append(1 if i <= 2310 else 0)

# Pad sequences for uniformity
data = pad_sequences(data, maxlen=max_length, dtype='float32')

labels = np.array(labels)
data = np.expand_dims(data, axis=-1)  # Add a new dimension

# Now you should be able to get a 3D shape from each element in data
print("Shape of the first reshaped item in data:", data[0].shape)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

# Convert labels to one-hot encoding
num_classes = len(np.unique(labels))
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

# Define your CNN model
model = Sequential()
model.add(Conv1D(32, kernel_size=3, activation='relu', input_shape=(max_length, 1)))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(64, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

# Compile and train the model
model.compile(loss=categorical_crossentropy, optimizer=Adam(), metrics=['accuracy'])
model.fit(X_train, y_train, batch_size=32, epochs=10, verbose=1, validation_data=(X_test, y_test))

# Evaluate the model
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])


0_mgdcc.pt
1_mgdcc.pt
2_mgdcc.pt
3_mgdcc.pt
4_mgdcc.pt
5_mgdcc.pt
6_mgdcc.pt
7_mgdcc.pt
8_mgdcc.pt
9_mgdcc.pt
10_mgdcc.pt
11_mgdcc.pt
12_mgdcc.pt
13_mgdcc.pt
14_mgdcc.pt
15_mgdcc.pt
16_mgdcc.pt
17_mgdcc.pt
18_mgdcc.pt
19_mgdcc.pt
20_mgdcc.pt
21_mgdcc.pt
22_mgdcc.pt
23_mgdcc.pt
24_mgdcc.pt
25_mgdcc.pt
26_mgdcc.pt
27_mgdcc.pt
28_mgdcc.pt
29_mgdcc.pt
30_mgdcc.pt
31_mgdcc.pt
32_mgdcc.pt
33_mgdcc.pt
34_mgdcc.pt
35_mgdcc.pt
36_mgdcc.pt
37_mgdcc.pt
38_mgdcc.pt
39_mgdcc.pt
40_mgdcc.pt
41_mgdcc.pt
42_mgdcc.pt
43_mgdcc.pt
44_mgdcc.pt
45_mgdcc.pt
46_mgdcc.pt
47_mgdcc.pt
48_mgdcc.pt
49_mgdcc.pt
50_mgdcc.pt
51_mgdcc.pt
52_mgdcc.pt
53_mgdcc.pt
54_mgdcc.pt
55_mgdcc.pt
56_mgdcc.pt
57_mgdcc.pt
58_mgdcc.pt
59_mgdcc.pt
60_mgdcc.pt
61_mgdcc.pt
62_mgdcc.pt
63_mgdcc.pt
64_mgdcc.pt
65_mgdcc.pt
66_mgdcc.pt
67_mgdcc.pt
68_mgdcc.pt
69_mgdcc.pt
70_mgdcc.pt
71_mgdcc.pt
72_mgdcc.pt
73_mgdcc.pt
74_mgdcc.pt
75_mgdcc.pt
76_mgdcc.pt
77_mgdcc.pt
78_mgdcc.pt
79_mgdcc.pt
80_mgdcc.pt
81_mgdcc.pt
82_mgdcc.pt
83_mgdcc.pt
84

In [15]:
import os
import numpy as np
import torch
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import categorical_crossentropy
from sklearn.model_selection import train_test_split

directory = r'F:\data_mfmgdcc'  # Replace with the directory path containing the .pt files

data = []
labels = []
max_length = 0

for i in range(5993):
    filename = f'{i}_mfmgdcc.pt'
    if filename in os.listdir(directory):  # Ensure the file exists in the directory
        feature = torch.load(os.path.join(directory, filename))
        feature = feature.view(-1)  # reshape to 1D tensor
        data.append(feature.numpy())
        max_length = max(max_length, feature.shape[0])
        print(filename)
        labels.append(1 if i <= 2310 else 0)

# Pad sequences for uniformity
data = pad_sequences(data, maxlen=max_length, dtype='float32')

labels = np.array(labels)
data = np.expand_dims(data, axis=-1)  # Add a new dimension

# Now you should be able to get a 3D shape from each element in data
print("Shape of the first reshaped item in data:", data[0].shape)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

# Convert labels to one-hot encoding
num_classes = len(np.unique(labels))
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

# Define your CNN model
model = Sequential()
model.add(Conv1D(32, kernel_size=3, activation='relu', input_shape=(max_length, 1)))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(64, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

# Compile and train the model
model.compile(loss=categorical_crossentropy, optimizer=Adam(), metrics=['accuracy'])
model.fit(X_train, y_train, batch_size=32, epochs=10, verbose=1, validation_data=(X_test, y_test))

# Evaluate the model
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])


0_mfmgdcc.pt
1_mfmgdcc.pt
2_mfmgdcc.pt
3_mfmgdcc.pt
4_mfmgdcc.pt
5_mfmgdcc.pt
6_mfmgdcc.pt
7_mfmgdcc.pt
8_mfmgdcc.pt
9_mfmgdcc.pt
10_mfmgdcc.pt
11_mfmgdcc.pt
12_mfmgdcc.pt
13_mfmgdcc.pt
14_mfmgdcc.pt
15_mfmgdcc.pt
16_mfmgdcc.pt
17_mfmgdcc.pt
18_mfmgdcc.pt
19_mfmgdcc.pt
20_mfmgdcc.pt
21_mfmgdcc.pt
22_mfmgdcc.pt
23_mfmgdcc.pt
24_mfmgdcc.pt
25_mfmgdcc.pt
26_mfmgdcc.pt
27_mfmgdcc.pt
28_mfmgdcc.pt
29_mfmgdcc.pt
30_mfmgdcc.pt
31_mfmgdcc.pt
32_mfmgdcc.pt
33_mfmgdcc.pt
34_mfmgdcc.pt
35_mfmgdcc.pt
36_mfmgdcc.pt
37_mfmgdcc.pt
38_mfmgdcc.pt
39_mfmgdcc.pt
40_mfmgdcc.pt
41_mfmgdcc.pt
42_mfmgdcc.pt
43_mfmgdcc.pt
44_mfmgdcc.pt
45_mfmgdcc.pt
46_mfmgdcc.pt
47_mfmgdcc.pt
48_mfmgdcc.pt
49_mfmgdcc.pt
50_mfmgdcc.pt
51_mfmgdcc.pt
52_mfmgdcc.pt
53_mfmgdcc.pt
54_mfmgdcc.pt
55_mfmgdcc.pt
56_mfmgdcc.pt
57_mfmgdcc.pt
58_mfmgdcc.pt
59_mfmgdcc.pt
60_mfmgdcc.pt
61_mfmgdcc.pt
62_mfmgdcc.pt
63_mfmgdcc.pt
64_mfmgdcc.pt
65_mfmgdcc.pt
66_mfmgdcc.pt
67_mfmgdcc.pt
68_mfmgdcc.pt
69_mfmgdcc.pt
70_mfmgdcc.pt
71_mfmgdcc.pt
72

In [ ]:
import os
import numpy as np
import torch
import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from sklearn.model_selection import train_test_split
from keras.layers import Conv1D, MaxPooling1D

directory = r'F:\data_mfmgdcc'  # Replace with the directory path containing the .pt files

data = []
labels = []



for i in range(5993):
    filename = f'{i}_mfmgdcc.pt'
    if filename in os.listdir(directory):  # Ensure the file exists in the directory
        feature = torch.load(os.path.join(directory, filename))
        print(filename)
        # Reshape the feature to a 2D array with a single row and 40 columns
        data.append(feature.view(1, -1).numpy())
        # Assign labels: first 2312 files are labelled 1, rest are labelled 0
        labels.append(1 if i <= 2310 else 0)


labels = np.array(labels)
data  = np.array(data)

# Now you should be able to get a 3D shape from each element in data
print("Shape of the first reshaped item in data:", data[0].shape)

# Split the data into training and testing sets
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

# Reshape the input data to match the CNN input shape
input_shape = (data[0].shape[1], 1)

X_train = X_train.reshape(X_train.shape[0], X_train.shape[2], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[2], 1)
# Convert labels to one-hot encoding
num_classes = len(np.unique(labels))
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

# Now you can define and train your CNN as before
# ...




# Create the CNN model


model = Sequential()
model.add(Conv1D(32, kernel_size=3, activation='relu', input_shape=input_shape))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(64, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

# Compile and train the model
model.compile(loss=keras.losses.categorical_crossentropy, optimizer=keras.optimizers.Adam(), metrics=['accuracy'])
model.fit(X_train, y_train, batch_size=32, epochs=10, verbose=1, validation_data=(X_test, y_test))

# Evaluate the model
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])


In [14]:
import os
import numpy as np
import torch
from keras.models import Sequential
from keras.layers import Bidirectional, LSTM, Dropout, Dense
from sklearn.model_selection import train_test_split

directory = r'F:\data_whisper2'  # Replace with the directory path containing the .pt files

data = []
labels = []

for i in range(5993):
    filename = f'reduced_{i}_whisper.pt'
    if filename in os.listdir(directory):  # Ensure the file exists in the directory
        feature = torch.load(os.path.join(directory, filename))
        print(filename)
        data.append(feature.view(-1, 1).numpy())  # Adjusted this line
        labels.append(1 if i <= 2310 else 0)
        

labels = np.array(labels)
data  = np.array(data)
num_classes = len(np.unique(labels))
print("Shape of the first reshaped item in data:", data[0].shape)

X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

print("Shape of labels: ", labels.shape)  # It should be (n_samples, n_classes)

def get_model(op_nodes):
    model = Sequential()
    model.add(Bidirectional(LSTM(units=32, return_sequences=True), input_shape=(data[0].shape[0], data[0].shape[1])))
    model.add(Dropout(0.1))
    model.add(Bidirectional(LSTM(units=32)))
    model.add(Dropout(0.1))
    model.add(Dense(op_nodes, activation='softmax'))

    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    return model

# Create the BiLSTM model
model = get_model(num_classes)

# Train the model
model.fit(X_train, y_train, batch_size=32, epochs=10, verbose=1, validation_data=(X_test, y_test))

# Evaluate the model
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])


reduced_0_whisper.pt
reduced_1_whisper.pt
reduced_2_whisper.pt
reduced_3_whisper.pt
reduced_4_whisper.pt
reduced_5_whisper.pt
reduced_6_whisper.pt
reduced_7_whisper.pt
reduced_8_whisper.pt
reduced_9_whisper.pt
reduced_10_whisper.pt
reduced_11_whisper.pt
reduced_12_whisper.pt
reduced_13_whisper.pt
reduced_14_whisper.pt
reduced_15_whisper.pt
reduced_16_whisper.pt


reduced_17_whisper.pt
reduced_18_whisper.pt
reduced_19_whisper.pt
reduced_20_whisper.pt
reduced_21_whisper.pt
reduced_22_whisper.pt
reduced_23_whisper.pt
reduced_24_whisper.pt
reduced_25_whisper.pt
reduced_26_whisper.pt
reduced_27_whisper.pt
reduced_28_whisper.pt
reduced_29_whisper.pt
reduced_30_whisper.pt
reduced_31_whisper.pt
reduced_32_whisper.pt
reduced_33_whisper.pt
reduced_34_whisper.pt
reduced_35_whisper.pt
reduced_36_whisper.pt
reduced_37_whisper.pt
reduced_38_whisper.pt
reduced_39_whisper.pt
reduced_40_whisper.pt
reduced_41_whisper.pt
reduced_42_whisper.pt
reduced_43_whisper.pt
reduced_44_whisper.pt
reduced_45_whisper.pt
reduced_46_whisper.pt
reduced_47_whisper.pt
reduced_48_whisper.pt
reduced_49_whisper.pt
reduced_50_whisper.pt
reduced_51_whisper.pt
reduced_52_whisper.pt
reduced_53_whisper.pt
reduced_54_whisper.pt
reduced_55_whisper.pt
reduced_56_whisper.pt
reduced_57_whisper.pt
reduced_58_whisper.pt
reduced_59_whisper.pt
reduced_60_whisper.pt
reduced_61_whisper.pt
reduced_62

In [5]:
pip install datasets

                                              0.0/486.2 kB ? eta -:--:--
     ---------                              122.9/486.2 kB 3.6 MB/s eta 0:00:01
     -------------------                    245.8/486.2 kB 3.0 MB/s eta 0:00:01
     ------------------------------         389.1/486.2 kB 2.7 MB/s eta 0:00:01
     -------------------------------------- 486.2/486.2 kB 2.8 MB/s eta 0:00:00
                                              0.0/21.5 MB ? eta -:--:--
                                              0.2/21.5 MB 4.6 MB/s eta 0:00:05
                                              0.3/21.5 MB 3.4 MB/s eta 0:00:07
                                              0.5/21.5 MB 3.9 MB/s eta 0:00:06
     -                                        0.7/21.5 MB 3.9 MB/s eta 0:00:06
     -                                        0.9/21.5 MB 4.1 MB/s eta 0:00:06
     --                                       1.1/21.5 MB 4.5 MB/s eta 0:00:05
     --                                       1.3/21.5 MB 4.5

In [ ]:
from transformers import WhisperProcessor, WhisperForConditionalGeneration
from datasets import load_dataset
from transformers.models.whisper.modeling_whisper import WhisperModel, WhisperEncoder
from transformers.models.whisper.configuration_whisper import WhisperConfig
from typing import Optional, Tuple, Union
import torch
import librosa 
import matplotlib.pyplot as plt
import numpy as np
import os, glob, pickle
import scipy.io as sio